15/03/2025

In [1]:
import pandas as pd

In [2]:
df_clienti = pd.read_csv(r"C:\Users\enduser\Desktop\PythonPostLaurea\ZOPPIS\ZOPPIS\data\clienti.csv",sep = ";")


### Full text creation

In [3]:
import ollama
import json
import os

In [4]:
def create_full_text(row, prefix="search_document"):
    return f"{prefix}: " + ", ".join(
        f"{col}: {val}"
        for col, val in row.items() 
        if pd.notna(val)
    )

df_clienti["full_text"] = df_clienti.apply(create_full_text, axis=1)

In [5]:
df_clienti_full_text = df_clienti["full_text"]

### Embedding generation

NB: sulla repo di nomic-embed-text dicono che bisogna mettere una chiave per dire che tipo di task si vuole fare.

In [22]:
import google.generativeai as genai
import os
import base64
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.environ.get("GOOGLE_API_KEY")

In [23]:
# import google.generativeai as genai

# genai.configure(api_key=API_KEY)

# models = genai.list_models()

# for model in models:
#     print(model.name)


In [101]:
import os
import pandas as pd
import ollama
import google.generativeai as genai
import time

def get_embedding(text, method="local", task_type = "retrieval_document"):
    if method == "local":
        try:
            response = ollama.embeddings(model="nomic-embed-text", prompt=text)
            return response["embedding"]
        except Exception as e:
            print(f"Errore durante l'embedding: {e}")
            return None
    if method == "api":
        try:
            #time.sleep(2)
            response = genai.embed_content(
                model="models/text-embedding-004",
                content=text,
                task_type=task_type
            )
            return response["embedding"]
            
        except Exception as e:
            print(f"Errore durante l'embedding: {e}")
            return None

path = r"C:\Users\enduser\Desktop\PythonPostLaurea\ZOPPIS\ZOPPIS\embeddings\gemini_text_embedding_004.parquet"

if not os.path.exists(path):
    df_clienti_emb = df_clienti_full_text.apply(get_embedding, method="api")
    df_clienti_emb = pd.DataFrame(df_clienti_emb)
    df_clienti_emb.to_parquet(path, engine="pyarrow")
    print("Embeddings salvati")
else:
    print(f"Il path '{path}' esiste già, uscita dal ciclo.")


Il path 'C:\Users\enduser\Desktop\PythonPostLaurea\ZOPPIS\ZOPPIS\embeddings\gemini_text_embedding_004.parquet' esiste già, uscita dal ciclo.


In [84]:
df_clienti_emb = pd.read_parquet(path)

ci ha messo 13 min x 6600 clienti.

la dimensione è di 768

### OCR:

con gemini flash

In [141]:
import google.generativeai as genai
import os
import base64
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.environ.get("GOOGLE_API_KEY")
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("gemini-2.0-flash")
doc_path = r"C:\Users\enduser\Desktop\PythonPostLaurea\ZOPPIS\ZOPPIS\data\inputs\moam.pdf"
with open(doc_path, "rb") as doc_file:
    doc_data = base64.standard_b64encode(doc_file.read()).decode("utf-8")
prompt = """
Trascrivi fedelmente il contenuto di questo documento PDF. Non aggiungere spiegazioni, riassunti o commenti.
Devi solo riportare il testo esattamente come appare nel documento.
"""
response = model.generate_content([
    {'mime_type': 'application/pdf', 'data': doc_data},
    prompt
])
#print(response.text)
ordine = response.text


In [142]:
ordine

"ORDINE MOAM 13/12/24 Milano\nMittente Erik Calati <erikcalati@gmail.com>\nDestinatario Zoppis Srl-ordini <ordini@zoppis.com>\nCc <flavio.allaria@gmail.com>\nData 2024-12-13 18:14\n\nBuona sera in allegato l'ordine in oggetto:\n• 2 box asahi 330 ml\n• 3box nastro azzurro 330 ml\n• 3 box Thomas Henry pompelmo rosa\n• 1 box coca cola 200 ml\n• 1 box coca cola 0 200 ml\n• 1 box Espolon Blanco\n• 1 box Campari\n• 2 bt Maraschino\n• 1 Punt e mes\n• 1 Ketel One Vodka\n• 1 box achillea arancia 200 ml\n• 1 box achillea pera 200 ml\n\nGrazie, Erik\n"

In [150]:
ordine = "ORDINE per il cliente MOAM di stresa, golf club alpino 13/12/24. ristorante Paolo Boneschi. Milano\nMittente Erik Calati <erikcalati@gmail.com>\nDestinatario Zoppis Srl-ordini <ordini@zoppis.com>\nCc <flavio.allaria@gmail.com>\nData 2024-12-13 18:14\n\nBuona sera in allegato l'ordine in oggetto:\n• 2 box asahi 330 ml\n• 3box nastro azzurro 330 ml\n• 3 box Thomas Henry pompelmo rosa\n• 1 box coca cola 200 ml\n• 1 box coca cola 0 200 ml\n• 1 box Espolon Blanco\n• 1 box Campari\n• 2 bt Maraschino\n• 1 Punt e mes\n• 1 Ketel One Vodka\n• 1 box achillea arancia 200 ml\n• 1 box achillea pera 200 ml\n\nGrazie, Erik\n"

### RAG

In [ ]:
path = r"C:\Users\enduser\Desktop\PythonPostLaurea\ZOPPIS\ZOPPIS\embeddings\gemini_text_embedding_004.parquet"

df_clienti_emb = pd.read_parquet(path)
query = "search_query: Trovami l'IDCliente (Codice Identificatore Cliente) del seguente ordine: \n" + ordine
query_emb = get_embedding(query, task_type="retrieval_query", method="api")

In [159]:
import numpy as np
from scipy.spatial.distance import cdist
k = 15
embeddings_series = df_clienti_emb["full_text"]
query_embedding = query_emb  
embedding_matrix = np.array(embeddings_series.tolist(), dtype=np.float32)
print("Shape della matrice degli embedding:", embedding_matrix.shape)
print("Shape dell'embedding della query:", len(query_embedding))
similarities = 1 - cdist([query_embedding], embedding_matrix, metric="cosine")[0]
top_k_indices = np.argsort(similarities)[::-1][:k]
print("Top k Indici più simili:", top_k_indices)
print("Similarità corrispondenti:", similarities[top_k_indices])

rag_results = "\n".join(df_clienti_full_text.loc[top_k_indices].astype(str).tolist())

Shape della matrice degli embedding: (6620, 768)
Shape dell'embedding della query: 768
Top k Indici più simili: [4054 4688 3281 3840 3873 5942 4245 1710 1576 2061 2597 3089 6226 3914
 2065]
Similarità corrispondenti: [0.7871963  0.76671059 0.76107913 0.75556207 0.75427078 0.75357112
 0.75231406 0.75210652 0.75146088 0.75030701 0.7499963  0.74902284
 0.74866227 0.74787257 0.74730569]


In [160]:
rag_results = "\n".join(df_clienti_full_text.loc[top_k_indices].astype(str).tolist())
prompt_rag = f"""search_query: Trovami l'IDCliente (Codice Identificatore Cliente) del seguente ordine:

🚀 **IMPORTANTE: INIZIO ORDINE** 🚀

{ordine}

 **IMPORTANTE: FINE ORDINE**

 "sapendo che tramite RAG ho ottenuto i full text (stringa contenente tutti i dati strutturati tabellari) dei clienti più simili all'ordine in oggetto. Riporto di seguito i dati full text dei clienti più simili all'ordine. Dai importanza sopratutto alla destinazione dell'ordine e alla ragione sociale dell'ordine (l'Indirizzo Dest la RagSoc nella tabella) per scegliere la riga trai top-k che ti propongo! \n"
  
 🚀 **IMPORTANTE: INIZIO ELENCO DATI ESTRATTI TRAMITE RAG** 🚀
 
 {rag_results}

  **IMPORTANTE: FINE ELENCO DATI ESTRATTI TRAMITE RAG**

"\nRestituiscimi soltanto l'IDCliente, la RagSoc e l'Indirizzo Dest in formato JSON, non voglio alcun tipo di dettaglio aggiuntivo"" 
"""

In [161]:
print(prompt_rag)

search_query: Trovami l'IDCliente (Codice Identificatore Cliente) del seguente ordine:

🚀 **IMPORTANTE: INIZIO ORDINE** 🚀

ORDINE per il cliente MOAM di stresa, golf club alpino 13/12/24. ristorante Paolo Boneschi. Milano
Mittente Erik Calati <erikcalati@gmail.com>
Destinatario Zoppis Srl-ordini <ordini@zoppis.com>
Cc <flavio.allaria@gmail.com>
Data 2024-12-13 18:14

Buona sera in allegato l'ordine in oggetto:
• 2 box asahi 330 ml
• 3box nastro azzurro 330 ml
• 3 box Thomas Henry pompelmo rosa
• 1 box coca cola 200 ml
• 1 box coca cola 0 200 ml
• 1 box Espolon Blanco
• 1 box Campari
• 2 bt Maraschino
• 1 Punt e mes
• 1 Ketel One Vodka
• 1 box achillea arancia 200 ml
• 1 box achillea pera 200 ml

Grazie, Erik


 **IMPORTANTE: FINE ORDINE**

 "sapendo che tramite RAG ho ottenuto i full text (stringa contenente tutti i dati strutturati tabellari) dei clienti più simili all'ordine in oggetto. Riporto di seguito i dati full text dei clienti più simili all'ordine. Dai importanza sopratutto a

In [162]:
def count_tokens_simple(text):
    return len(text.split())

print(count_tokens_simple(prompt_rag))


915


In [163]:
generation_config = {
  "temperature": 0.1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "application/json", # posso richiedere esplicitamente il formato strutturato della risposta
}

model = genai.GenerativeModel(
    model_name="gemini-2.0-flash-exp",
    generation_config=generation_config,
    #system_instruction="Sei un venditore, quindi i campi vuoti non li sostituisci con Null ma con valori inventati!" # solo come esperimento..
)

output_rag = model.generate_content(prompt_rag)
print(output_rag.text)

[
  {
    "IDCliente": "11036",
    "RagSoc": "PANETTERIA ZOPPIS SRL",
    "Indirizzo_Dest": "VIA A. DE GASPERI 12"
  }
]
